In [1]:
import xarray as xr
import xroms
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cmocean.cm as cmo
import pandas as pd

# How to calculate with `xroms`

Here we demonstrate a number of calculations built into `xroms`, through accessors to `DataArrays` and `Datasets`.

## `xarray` Datasets
Use an `xarray` accessor in `xroms` to easily perform calculations with syntax 

    ds.xroms.[method]
    
Importantly, the `xroms` accessor to a `Dataset` is initialized with an `xgcm` grid object, stored at `ds.xroms.grid`, which is used to perform the basic grid calculations. More on this under "How to set up grid" below.

Generally the built-in calculations take in the horizontal then vertical grid label you want the calculation to be on as options:

    ds.xroms.dudz('rho', 's_rho')  # to make sure result is on rho horizontal grid and s_rho vertical grid
    
or

    ds.xroms.dudz()  # return on whatever grid it is calculated on
    
These are not stored as properties in the accessor since they depend on the user's choice of grids, but since they are lazily evaluated, they are quick to rerun as needed.

Other inputs are available for functions when the calculation involves a derivative and there is a choice for how to treat the boundary (`hboundary` and `hfill_value` for horizontal calculations and `sboundary` and `sfill_value` for vertical calculations). More information on those inputs can be found in the docs for `xgcm` such as under found under:

    ds.xroms.grid.interp?


### `xarray` DataArrays

A few of the more basic methods in `xroms` are available to `DataArrays` too. Unlike a `Dataset` in `xroms`, a `DataArray` does not know about its grid (a `Dataset` has its grid information stored in `ds.xroms.grid`). So, the built-in `xroms` methods for `DataArrays` require the grid object to be input.

    ds.temp.xroms.to_grid(ds.xroms.grid, hcoord='psi', scoord='s_w')

### Load in data

More information at in [load_data notebook](load_data.ipynb)

In [2]:
loc = 'http://barataria.tamu.edu:8080/thredds/dodsC/forecast_latest/txla2_his_f_latest.nc'
chunks = {'ocean_time':1}
ds = xroms.open_netcdf(loc, chunks=chunks)  # also adds z coordinates

## How to set up grid

The package `xcgm` has many nice grid functions for ROMS users, however, a bit of set up is required to connect from ROMS to MITgcm output. This grid set up does that.

The `grid` object contains metrics (X, Y, Z) with distances for each grid ('dx', 'dx_u', 'dx_v', 'dx_psi', and 'dz', 'dz_u', 'dz_v', 'dz_w', 'dz_w_u', 'dz_w_v', 'dz_psi', 'dz_w_psi'), and all of these as grid coordinates too. 

Coordinates are added to the dataset when `xroms.open_netcdf` or `xroms.open_zarr` are used. The grid object is read into the `xroms` accessor and saved in the location `ds.xroms.grid`.

You can also explicitly set up the grid with:

> ds, grid = xroms.roms_dataset(ds)

## Basic functionality

### Change grids

A ROMS user frequently needs to move between horizontal and vertical grids, so it is built into many of the function wrappers, but you can also do it as a separate function. It can also be done to both `Datasets` and `DataArrays` with slightly different syntax. Here we change salinity from its default grids to be on the psi grid horizontally and the s_w grid vertically:

In [3]:
ds.xroms.to_grid('salt', 'psi', 's_w');   # Dataset
ds.salt.xroms.to_grid(ds.xroms.grid, 'psi', 's_w');   # DataArray

You can also go to the original `xroms` function and avoid the `xarray` accessor if you prefer, though the point of the accessor approach (that is, `ds.xroms...`) is to be easier to remember and less code to write generally.

In [4]:
xroms.to_grid(ds.salt, ds.xroms.grid, 'psi', 's_w');

## Calculations

### Vertical derivative, d/dz

Take a vertical derivative of a variable:

In [5]:
ds.xroms.ddz('salt');  # Dataset

In [6]:
ds.salt.xroms.ddz(ds.xroms.grid);  # DataArray

In [7]:
xroms.ddz(ds.salt, ds.xroms.grid);  # No accessor

A convenience function `calc_ddz` is also available to wrap `ddz` along with `to_grid` so that calculating a vertical derivative and changing grids can occur in one line:

In [8]:
ds.xroms.calc_ddz('salt', 'dsaltdz', hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # Dataset

In [9]:
ds.salt.xroms.calc_ddz(ds.xroms.grid, 'dsaltdz', hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # DataArray

In [10]:
xroms.calc_ddz(ds.salt, ds.xroms.grid, 'dsaltdz', hcoord='psi', scoord='s_rho', sboundary='extend', sfill_value=np.nan);  # No accessor

### Vertical shear

Since it is a common use case, there are specific methods to return the u and v components of vertical shear on their own grids, as follows, and note that as always these functions will take in `hcoord` and `scoord` to shift grids. These are just available for Datasets.

In [11]:
ds.xroms.dudz();  

In [12]:
ds.xroms.dvdz();

If we want to calculate something with both, we need them on the same grid. For this, we can input the desired resultant grid:

In [13]:
ds.xroms.dudz(hcoord='rho', scoord='s_rho')**2 + ds.xroms.dvdz(hcoord='rho', scoord='s_rho')**2;

### density

In [14]:
ds.xroms.get_rho();

### buoyancy frequency

In [15]:
ds.xroms.N2(hcoord=None, scoord='s_w', sboundary='fill', sfill_value=np.nan);

### vertical vorticity

In [16]:
ds.xroms.vort();

### ertel

In [17]:
ds.xroms.ertel();